In [1]:
import numpy as np
File = open('Predicts.csv')
Raw = File.readlines()
File.close()
Raw.pop(0)

'Id,Prediction\n'

In [2]:
from collections import defaultdict
Data = defaultdict(list)
Number = {}

n = 0; old = ''
for line in Raw:
    ID,cha = line.strip().split(',')
    ID = '_'.join(ID.split('_')[:2])
    Data[ID].append(cha)
    if ID != old:
        Number[n] = ID
        n += 1
        old = ID
        
        
Data['fjmg0_sx11'], Number[0]

(['sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'sil',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'hh',
  'iy',
  'iy',
  'iy',
  'iy',
  'iy',
  'uw',
  'uw',
  'iy',
  'iy',
  'w',
  'w',
  'w',
  'w',
  'w',
  'l',
  'l',
  'l',
  'ih',
  'ih',
  'ih',
  'ih',
  'ih',
  'ih',
  'ow',
  'ow',
  'ow',
  'l',
  'l',
  'l',
  'l',
  'l',
  'ah',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'ah',
  'ah',
  'ow',
  'ah',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'l',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'ae',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aw',
  'aa',
  'aa',
  'aa',
  'aa',
  'ah',
  'ah',
  'l',
  'sil',
  'k',
  'aa',
  'aa',
  'aa',
  'aa',
  'aa',
  'aa',
  'aa',
  'er',
  'er',
  'er',
  'er',
  'r',
  'r',
  'r',
  'r',
  'r',
  'r',
  'r',
  'r'

In [3]:
def decide(value,i):
    if value[i-1] == value[i+1]:
        return True
    elif value[i]!= value[i+1]:
        return True
    else:
        return False
    
for key,value in Data.items():
    for i in range(1,len(value)-1):
        if decide(value,i) and value[i]!= value[i-1]:
            value[i] = value[i-1]
    Data[key] = value
    
Data['fjmg0_sx11']

['sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'sil',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'hh',
 'iy',
 'iy',
 'iy',
 'iy',
 'iy',
 'uw',
 'uw',
 'iy',
 'iy',
 'w',
 'w',
 'w',
 'w',
 'w',
 'l',
 'l',
 'l',
 'ih',
 'ih',
 'ih',
 'ih',
 'ih',
 'ih',
 'ow',
 'ow',
 'ow',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'ah',
 'ah',
 'ah',
 'ah',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'l',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aw',
 'aa',
 'aa',
 'aa',
 'aa',
 'ah',
 'ah',
 'ah',
 'ah',
 'ah',
 'aa',
 'aa',
 'aa',
 'aa',
 'aa',
 'aa',
 'aa',
 'er',
 'er',
 'er',
 'er',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'r',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'ae',
 'eh',
 'eh',
 'eh',
 'eh',
 'eh',
 'eh',
 'eh'

In [4]:
Pre = defaultdict(list)
for key,value in Data.items():
    for v in value:
        if len(Pre[key])==0 or Pre[key][-1]!= v:
            Pre[key].append(v)
            
Pre['fjmg0_sx11']

['sil',
 'hh',
 'iy',
 'uw',
 'iy',
 'w',
 'l',
 'ih',
 'ow',
 'l',
 'ah',
 'l',
 'ae',
 'aw',
 'aa',
 'ah',
 'aa',
 'er',
 'r',
 'ae',
 'eh',
 'er',
 'l',
 'ay',
 'er',
 'ay',
 'er',
 'sil']

In [5]:
M = open('48_idx_chr.map_b')
mapp = M.readlines()
M.close()

Map = {}
for line in mapp:
    line = line.strip().split(' ')
    word,_ = line[0].split('\t')
    Map[word] = line[-1]
Map

{'aa': 'a',
 'ae': 'b',
 'ah': 'c',
 'ao': 'd',
 'aw': 'e',
 'ax': 'f',
 'ay': 'g',
 'b': 'h',
 'ch': 'i',
 'cl': 'j',
 'd': 'k',
 'dh': 'l',
 'dx': 'm',
 'eh': 'n',
 'el': 'o',
 'en': 'p',
 'epi': 'q',
 'er': 'r',
 'ey': 's',
 'f': 't',
 'g': 'u',
 'hh': 'v',
 'ih': 'w',
 'ix': 'x',
 'iy': 'y',
 'jh': 'z',
 'k': 'A',
 'l': 'B',
 'm': 'C',
 'n': 'D',
 'ng': 'E',
 'ow': 'F',
 'oy': 'G',
 'p': 'H',
 'r': 'I',
 's': 'J',
 'sh': 'K',
 'sil': 'L',
 't': 'M',
 'th': 'N',
 'uh': 'O',
 'uw': 'P',
 'v': 'Q',
 'vcl': 'R',
 'w': 'S',
 'y': 'T',
 'z': 'U',
 'zh': 'V'}

In [7]:
import csv
csvfile = open('V1.csv','w',newline='')
write = csv.writer(csvfile, delimiter=',')
write.writerow(['id','phone_sequence'])

for i in range(len(Number)):
    S = ''
    for word in Pre[Number[i]]:
        S = S + Map[word]
    write.writerow([Number[i],S])

In [11]:
len(Number)

592